In [2]:
import time
import os
import json
from IPython.display import display, Markdown
from typing import List

import openai
from openai import OpenAI

In [3]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Integration OpenAI Beta Assistant

In [4]:
client = OpenAI(api_key="YOUR API KEY")

In [21]:
assistant = client.beta.assistants.create(
    name="Math tutor",
    instructions="You are a personal Math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo",
)
#show_json(assistant)

In [22]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_5KOcCvRjMUuu4WhVI2XVQbLO',
 'created_at': 1727716264,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [23]:

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Explain me why 1 + 1 = 2?",
)
#show_json(message)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
#show_json(run)

run = wait_on_run(run, thread)
#show_json(run)

messages = list(client.beta.threads.messages.list(thread_id=thread.id))
#show_json(messages)

print(messages[0].content[0].text.value)

In mathematics, the sum of 1 and 1 is 2 because when you combine one item with another item, you have a total of two items.


In [24]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Can you translate the last response to me in french?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = list(client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
))
#show_json(messages)

print(messages[0].content[0].text.value)

En mathématiques, la somme de 1 et 1 est égale à 2 car lorsque vous combinez un élément avec un autre élément, vous obtenez un total de deux éléments.


In [25]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="tell me what were the last two answers from you?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = list(client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
))
#show_json(messages)

print(messages[0].content[0].text.value)

My last two answers were explaining why 1 + 1 = 2 in mathematics, and then translating that explanation into French.


# Integration OpenAI Beta Vectorstore together with the OpenAI Beta Assistant

In [26]:
assistant = client.beta.assistants.create(
    name="File Search Expert",
    instructions="You are a personal File search expert. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo",
    tools=[{"type": "file_search"}]
)
show_json(assistant)

{'id': 'asst_ewV4u5rfzsMR6B9C0AvBOZuu',
 'created_at': 1727716375,
 'description': None,
 'instructions': 'You are a personal File search expert. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'File Search Expert',
 'object': 'assistant',
 'tools': [{'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'ranker': 'default_2024_08_21',
     'score_threshold': 0.0}}}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None,
  'file_search': {'vector_store_ids': []}},
 'top_p': 1.0}

In [27]:
# Step 1: Get the list of files from the user
file_paths = ['files/Article-1.pdf','files/CV_1.pdf']

In [28]:
# Step 2: Create a vector store and upload files
vector_store = client.beta.vector_stores.create(name="User Files")

In [29]:
# Prepare the files for upload
file_streams = [open(path, "rb") for path in file_paths]

# Upload files and add them to the vector store
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

In [30]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [31]:
# Step 3: Update the assistant to use the new Vector Store
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [32]:
thread = client.beta.threads.create()

In [33]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="What is article-1 about?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = list(client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
))
#show_json(messages)

print(messages[0].content[0].text.value)


Article-1 discusses the comparison between open-source Large Language Models (LLMs) and proprietary models like ChatGPT, focusing on factors like data protection, governance, and adoption considerations【4:0†source】.


In [34]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Translate the last response to french"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = list(client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
))
#show_json(messages)

print(messages[0].content[0].text.value)


L'article 1 discute de la comparaison entre les grands modèles de langage à source ouverte (LLMs) et les modèles propriétaires comme ChatGPT, en se concentrant sur des facteurs tels que la protection des données, la gouvernance et les considérations d'adoption【4:0†source】.
